In [1]:
#import wget
#wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Reading the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Creating a dash application
app = dash.Dash(__name__)

# Creating an app layout
app.layout = html.Div(children=
                      [html.H1('SpaceX Launch Records Dashboard',
                               style={'textAlign': 'center',
                                      'color': '#503D36',
                                      'font-size': 40
                                     }
                              ),
                       
                       # dropdown list to enable Launch Site selection
                       dcc.Dropdown(id='site-dropdown',
                                    options=[{'label': 'Select a Launch Site here', 'value': 'placeholder', 'disabled': True},
                                             {'label': 'All Sites', 'value': 'ALL'}]
                                            +
                                            [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()],
                                    
                                    value='ALL' # The default select value is for All Sites
                                   ),
                       html.Br(),
                       
                       # Adding a pie chart to show the total successful launches count for all sites
                       # If a specific launch site was selected, showing the Success vs. Failed counts for the site
                       html.Div([],id='success-pie-chart'),
                       
                       html.Br(),
                       
                       html.P("Payload range (Kg):"),
                       
                       # Adding a slider to select payload range
                       dcc.RangeSlider(id='payload-slider',
                                       min=0,
                                       max=10000,
                                       step=1000,
                                       value=[min_payload,max_payload]
                                      ),
                       
                       # Adding a scatter chart to show the correlation between payload and launch success
                       html.Div([],id='success-payload-scatter-chart'),
                      ])

# Adding a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback([Output(component_id='success-pie-chart',component_property='children')],
              [Input(component_id='site-dropdown',component_property='value')]
             )
def pie_chart (site):
    if site=='ALL':
        pie_fig=px.pie(spacex_df[spacex_df['class']==1]['Launch Site'].value_counts().reset_index(),
                       values='count',
                       names='Launch Site',
                       title='Total Success Launches by Site')
    else:
        pie_fig=px.pie(spacex_df[spacex_df['Launch Site']==site]['class'].value_counts().reset_index(),
                       values='count',
                       names='class',
                       title=f'Total Success Launches for site {site}'
                      )
    return [dcc.Graph(figure=pie_fig)]
    

# Adding a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback([Output(component_id='success-payload-scatter-chart',component_property='children')],
              [Input(component_id='site-dropdown',component_property='value'),
               Input(component_id='payload-slider',component_property='value')]
             )
def scatter_chart (site,payload):
    if site=='ALL':
        scatter_fig=px.scatter(spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])],
                               x='Payload Mass (kg)',
                               y='class',
                               color='Booster Version Category',
                               title='Correlation between Payload & Success for all Sites'
                              )
    else:
        scatter_fig=px.scatter(spacex_df[(spacex_df['Payload Mass (kg)'].between(payload[0],payload[1]))& (spacex_df['Launch Site']==site)],
                               x='Payload Mass (kg)',
                               y='class',
                               color='Booster Version Category',
                               title=f'Correlation between Payload & Success for site {site}'
                              )
    return [dcc.Graph(figure=scatter_fig)]

# Run the app
if __name__ == '__main__':
    app.run_server()


C:\Users\HP\AppData\Local\Temp\ipykernel_17400\2155157386.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\HP\AppData\Local\Temp\ipykernel_17400\2155157386.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [03/Nov/2023 02:15:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2023 02:15:30] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2023 02:15:30] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2023 02:15:30] "GET /_favicon.ico?v=2.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2023 02:15:30] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Nov/2023 02:15:30] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Nov/2023 02:15:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2023 02:15:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2023 02:15:31] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Nov/2023 02:15:31] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Nov/2023 02:15:37] "POST /_dash-update

1. Which site has the largest successful launches?
2. Which site has the highest launch success rate?
3. Which payload range(s) has the highest launch success rate?
4. Which payload range(s) has the lowest launch success rate?
5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest
launch success rate?
